# Notebook for End to End testing in the MID ITF

- Uses digitizers for the 2 inputs to the correlator
- Uses 1 FSP (200 MHz)
- Receives and plots the visibilities to SDP

In [1]:
import os
from time import sleep

print("Setting up parameters\n")

CLUSTER_DOMAIN = "miditf.internal.skao.int"

#SKA001_NAMESPACE = "ci-dish-lmc-ska001-at-2111-update-tmc"
#SKA036_NAMESPACE = "ci-dish-lmc-ska036-at-2111-update-tmc"
#SUT_NAMESPACE = "ci-ska-mid-itf-at-2111-update-tmc"

SKA001_NAMESPACE = "staging-dish-lmc-ska001" #ci-dish-lmc-ska001-at-2139-tmc-0-20-1-integration
SKA036_NAMESPACE = "staging-dish-lmc-ska036" #ci-dish-lmc-ska036-at-2139-tmc-0-20-1-integration
SUT_NAMESPACE = "staging" #"staging"

TANGO_HOST = f"tango-databaseds.{SUT_NAMESPACE}.svc.{CLUSTER_DOMAIN}:10000"
os.environ["TANGO_HOST"] = TANGO_HOST

RECEPTORS = ["SKA001", "SKA036"]

# Config files set up
DATA_DIR = "../../data"
TMC_CONFIGS = f"{DATA_DIR}/mid_telescope/tmc"
SCAN_FILE = f"{TMC_CONFIGS}/scan.json"
RELEASE_RESOURCES_FILE = f"{DATA_DIR}/release_resources.json"

ASSIGN_RESOURCES_FILE = f"{TMC_CONFIGS}/assign_resources.json"
CONFIGURE_SCAN_FILE = f"{TMC_CONFIGS}/configure_scan.json"

CBF_CONFIGS = f"{DATA_DIR}/mid_telescope/cbf"
DISH_CONFIG_FILE = f"{CBF_CONFIGS}/sys_params/load_dish_config.json"

KAFKA_PORT = 9092
KAFKA_SERVICE_NAME = "ska-sdp-kafka"
KAFKA_ENDPOINT = f"{KAFKA_SERVICE_NAME}.{SUT_NAMESPACE}.svc.{CLUSTER_DOMAIN}:{KAFKA_PORT}"

print("Links for Taranta and QA Display\n")
print("SUT Links")
print(f"https://k8s.{CLUSTER_DOMAIN}/{SUT_NAMESPACE}/signal/display/")
print(f"https://k8s.{CLUSTER_DOMAIN}/{SUT_NAMESPACE}/taranta/devices")
print("\n")
print("Dish LMC links")
print(f"https://k8s.{CLUSTER_DOMAIN}/{SKA001_NAMESPACE}/taranta/devices")
print(f"https://k8s.{CLUSTER_DOMAIN}/{SKA036_NAMESPACE}/taranta/devices")

Setting up parameters

Links for Taranta and QA Display

SUT Links
https://k8s.miditf.internal.skao.int/staging/signal/display/
https://k8s.miditf.internal.skao.int/staging/taranta/devices


Dish LMC links
https://k8s.miditf.internal.skao.int/staging-dish-lmc-ska001/taranta/devices
https://k8s.miditf.internal.skao.int/staging-dish-lmc-ska036/taranta/devices


### Set Up and Configure Device Proxies 

In [2]:
import json

from tango import DeviceProxy

print("Setting up the device proxies")

# TMC proxies
tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
tmc_csp_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")
tmc_csp_subarray = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
tmc_subarray = DeviceProxy("ska_mid/tm_subarray_node/1")

# CSP.LMC proxies
csp_control = DeviceProxy("mid-csp/control/0")
csp_subarray = DeviceProxy("mid-csp/subarray/01")

# CBF proxies
cbf_controller = DeviceProxy("mid_csp_cbf/sub_elt/controller")
cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")

# Dish Leaf Proxies
dish_leaf_node_ska001 = DeviceProxy("ska_mid/tm_leaf_node/d0001")
dish_leaf_node_ska036 = DeviceProxy("ska_mid/tm_leaf_node/d0036")

# SDP Proxies
sdp_subarray = DeviceProxy("mid-sdp/subarray/01")

# Set devices to adminMode = ONLINE
csp_control.adminMode = 0
csp_subarray.adminMode = 0

# Leaf Nodes
csp_subarray_leaf_node = "ska_mid/tm_leaf_node/csp_subarray01"
sdp_subarray_leaf_node = "ska_mid/tm_leaf_node/sdp_subarray01"
csp_master_leaf_node = "ska_mid/tm_leaf_node/csp_master"
csp_subarray_leaf_node_dp = DeviceProxy(csp_subarray_leaf_node)
sdp_subarray_leaf_node_dp = DeviceProxy(sdp_subarray_leaf_node)
csp_master_leaf_node_dp = DeviceProxy(csp_master_leaf_node)

sleep(2)
print("\nChecking admin mode after setting to ONLINE (0):")
print(f"  CSP Control: {csp_control.adminMode}")
print(f"  CSP Subarray: {csp_subarray.adminMode}")
print(f"  CBF Controller: {cbf_controller.adminMode}")
print(f"  CBF Subarray: {cbf_subarray.adminMode}")

# Set CBF Simulation mode to false and CBF timeout to 99s
csp_control.cbfSimulationMode = 0
csp_control.commandTimeout = 99

sleep(2)
print("\nChecking CBF Simulation Mode and CBF Timeout:")
print(f"  CBF Simulation Mode: {bool(csp_control.cbfSimulationMode)}")
print(f"  CBF Timeout: {csp_control.commandTimeout} sec")

Setting up the device proxies

Checking admin mode after setting to ONLINE (0):
  CSP Control: 0
  CSP Subarray: 0
  CBF Controller: 0
  CBF Subarray: 0

Checking CBF Simulation Mode and CBF Timeout:
  CBF Simulation Mode: False
  CBF Timeout: 99 sec


### Load the Dish Vcc Config / Init Sys Params

NOTE: RUN TWICE

In [4]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

dish_config_json["tm_data_sources"][0] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"

print(f"dish_config_json file contents: \n{dish_config_json}")
tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))

sleep(2)
print(f"TMC CSP Master's Dish Vcc Config attribute value: \n{tmc_csp_master.dishVccConfig}")
print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{tmc_csp_master.sourceDishVccConfig}"
)

dish_config_json file contents: 
{'interface': 'https://schema.skao.int/ska-mid-cbf-initsysparam/1.0', 'tm_data_sources': ['car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata'], 'tm_data_filepath': 'instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json'}
TMC CSP Master's Dish Vcc Config attribute value: 
{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "dish_parameters": {"SKA001": {"vcc": 1, "k": 1}, "SKA036": {"vcc": 2, "k": 1}, "SKA063": {"vcc": 3, "k": 1}, "SKA100": {"vcc": 4, "k": 1}}}

TMC CSP Master's Source Dish Vcc Config attribute value: 
{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "tm_data_sources": ["car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"], "tm_data_filepath": "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"}


### Turn the Telescope On
Known issues:
When turning on Telescope, talon DDR memory calibration can fail. An indicator of this is mid_csp_cbf/fspcoorrsubarray obsState = FAULT. To adress this:
* Manually power off of the PDUs to the talon boards
* Rerun of the Telescope ON command.

In [5]:
print("Running the TelescopeOn command")
tmc_central_node.TelescopeOn()

while int(tmc_central_node.telescopeState) != 0:
    print("Waiting for Telescope to come ON")
    sleep(5)
    
print("YAY Telescope is ON")


Running the TelescopeOn command
Waiting for Telescope to come ON
Waiting for Telescope to come ON
Waiting for Telescope to come ON
Waiting for Telescope to come ON
Waiting for Telescope to come ON
Waiting for Telescope to come ON
Waiting for Telescope to come ON
Waiting for Telescope to come ON
YAY Telescope is ON


In [6]:
print("Verifying the states:")
print(f"  TMC Central Node State: {tmc_central_node.State()}")
print(f"  CSP Control State: {csp_control.State()}")
print(f"  CBF Controller State: {cbf_controller.State()}")
print(f"  TMC Subarray State: {tmc_subarray.State()}")

Verifying the states:
  TMC Central Node State: ON
  CSP Control State: ON
  CBF Controller State: ON
  TMC Subarray State: ON


### Assign Resources
//Vis-receive pods comes up in the staging-sdp namespace 
Known issues:
1. The mid_csp_cbf/sub_elt/subarray_01 device times out. To resolve:
* Rerun the assign resources block

2. eb and pb ids already exist (logs in ds-subarraynode-01-0 pod). Resolution:
* Edit eb and pb ids in the assign_resources.json


In [7]:
print(sdp_subarray.state())

sleep(3)

print(
    "Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned"
)

with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    assign_resources_json = json.load(f)
    assign_resources_json["dish"]["receptor_ids"] = RECEPTORS
    assign_resources_json["sdp"]["resources"]["receptors"] = RECEPTORS
    assign_resources_json['sdp']['processing_blocks'][0]['parameters']['queue_connector_configuration']['exchanges'][0]['source']['servers'] = KAFKA_ENDPOINT
    assign_resources_json['sdp']['processing_blocks'][0]['parameters']['extra_helm_values']['receiver']['options']['reception']['stats_receiver_kafka_config'] = f"{KAFKA_ENDPOINT}:json_workflow_state"    

print(f"\nassign_resources_json file contents: \n{assign_resources_json}")

tmc_subarray.AssignResources(json.dumps(assign_resources_json))

sleep(2)
print(f"\nCBF Subarray Observation State: {tmc_subarray.obsState}")
print(f"CBF Subarray Receptors : {cbf_subarray.receptors}")

ON
Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned

assign_resources_json file contents: 
{'interface': 'https://schema.skao.int/ska-tmc-assignresources/2.1', 'transaction_id': 'txn-....-00001', 'subarray_id': 1, 'dish': {'receptor_ids': ['SKA001', 'SKA036']}, 'sdp': {'interface': 'https://schema.skao.int/ska-sdp-assignres/0.4', 'resources': {'csp_links': [1, 2, 3, 4], 'receptors': ['SKA001', 'SKA036'], 'receive_nodes': 1}, 'execution_block': {'eb_id': 'eb-test-20210630-00061', 'context': {}, 'max_length': 21600.0, 'channels': [{'channels_id': 'vis_channels', 'spectral_windows': [{'spectral_window_id': 'fsp_1_channels', 'count': 14880, 'start': 0, 'stride': 1, 'freq_min': 296000000.0, 'freq_max': 496000000.0, 'link_map': [[0, 0], [200, 1], [744, 2], [944, 3]]}]}], 'polarisations': [{'polarisations_id': 'all', 'corr_type': ['XX', 'XY', 'YX', 'YY']}], 'fields': [{'field_id': 'field_a', 'phase_dir': {'ra': [2.711325], 'dec': [-0

### Work around to slew the dishes to near the start of track to avoid tracking timing errors

Could take +1min

In [8]:
import time
SKA001_TANGO_HOST = f"tango-databaseds.{SKA001_NAMESPACE}.svc.{CLUSTER_DOMAIN}:10000"
dish_manager_ska001 = DeviceProxy(f"tango://{SKA001_TANGO_HOST}/mid-dish/dish-manager/ska001")
SKA036_TANGO_HOST = f"tango-databaseds.{SKA036_NAMESPACE}.svc.{CLUSTER_DOMAIN}:10000"
dish_manager_ska036 = DeviceProxy(f"tango://{SKA036_TANGO_HOST}/mid-dish/dish-manager/ska036")

dish_manager_ska001.slew([181.0,31.0])
dish_manager_ska036.slew([181.0,31.0])

while dish_manager_ska001.achievedpointing[1] != 181.0 or dish_manager_ska001.achievedpointing[2] != 31.0:
  time.sleep(1)

while dish_manager_ska036.achievedpointing[1] != 181.0 or dish_manager_ska036.achievedpointing[2] != 31.0:
  time.sleep(1)

print("Done slewing")

Done slewing


In [9]:
from astropy.time import Time
import time
from tango import DeviceProxy

dish_deployments = [f"tango://tango-databaseds.{SKA001_NAMESPACE}.svc.{CLUSTER_DOMAIN}:10000/mid-dish/dish-manager/SKA001", 
                    f"tango://tango-databaseds.{SKA036_NAMESPACE}.svc.{CLUSTER_DOMAIN}:10000/mid-dish/dish-manager/SKA036"]


def get_tai_from_unix_s(unix_s: float) -> float:
    """
    Calculate atomic time in seconds from unix time in seconds.

    :param unix_s: Unix time in seconds

    :return: atomic time (tai) in seconds
    """
    astropy_time_utc = Time(unix_s, format="unix")
    return astropy_time_utc.unix_tai


for dish in dish_deployments:
    dish_manager_proxy = DeviceProxy(dish)
    current_pointing = dish_manager_proxy.achievedPointing
    current_az = current_pointing[1]
    current_el = current_pointing[2]

    current_time_tai_s = get_tai_from_unix_s(time.time() + 120)

    # Directions to move values
    az_dir = 1 if current_az < 350 else -1
    el_dir = 1 if current_el < 80 else -1

    track_table = [
        current_time_tai_s + 3,
        current_az + 1 * az_dir,
        current_el + 1 * el_dir,
        current_time_tai_s + 5,
        current_az + 2 * az_dir,
        current_el + 2 * el_dir,
        current_time_tai_s + 7,
        current_az + 3 * az_dir,
        current_el + 3 * el_dir,
        current_time_tai_s + 9,
        current_az + 4 * az_dir,
        current_el + 4 * el_dir,
        current_time_tai_s + 11,
        current_az + 5 * az_dir,
        current_el + 5 * el_dir,
    ]

    dish_manager_proxy.programTrackTable = track_table

### Configure Scan

In [10]:
print("Running the Configure command: subarray obsstate should go to Ready")

with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    configure_scan_json = json.load(f)

print(f"\nconfigure_scan_json file contents: \n{configure_scan_json}")

print(json.dumps(configure_scan_json))

tmc_subarray.Configure(json.dumps(configure_scan_json))
    
sleep(12)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

Running the Configure command: subarray obsstate should go to Ready

configure_scan_json file contents: 
{'interface': 'https://schema.skao.int/ska-tmc-configure/2.0', 'pointing': {'target': {'reference_frame': 'ICRS', 'target_name': 'Polaris Australis', 'ra': '21:08:47.92', 'dec': '-88:57:22.9'}}, 'sdp': {'interface': 'https://schema.skao.int/ska-sdp-configure/0.4', 'scan_type': 'science'}, 'csp': {'interface': 'https://schema.skao.int/ska-csp-configure/2.4', 'subarray': {'subarray_name': 'Receptor(s)'}, 'common': {'config_id': '1 receptor, band 1, 2 FSP, no options', 'subarray_id': 1, 'frequency_band': '1'}, 'cbf': {'fsp': [{'fsp_id': 1, 'function_mode': 'CORR', 'frequency_slice_id': 1, 'zoom_factor': 0, 'integration_factor': 10, 'output_link_map': [[0, 1]], 'channel_offset': 0, 'zoom_window_tuning': 450000}]}}, 'tmc': {'scan_duration': 920.0}, 'dish': {'receiver_band': '1'}}
{"interface": "https://schema.skao.int/ska-tmc-configure/2.0", "pointing": {"target": {"reference_frame": "IC

In [11]:
print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")


CBF Subarray Observation State: 2


### Scan

DO NOT USE FOR NOW

In [ ]:
# print("Running the Scan command: subarray obsstate should go to Scanning")

# with open(SCAN_FILE, encoding="utf-8") as f:
#     scan_json = f.read()

# print(f"\nscan_json file contents: \n{scan_json}")

# tmc_subarray.Scan(scan_json)

# sleep(2)
# print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")

TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING

In [12]:
with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    sdp_scan = d["sdp"]
    print(sdp_scan)
    sdp_subarray_leaf_node_dp.scan(json.dumps(sdp_scan))

sleep(10)
print(sdp_subarray_leaf_node_dp.sdpSubarrayObsState)

with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_scan = d["csp"]
    print(csp_scan)
    csp_subarray_leaf_node_dp.scan(json.dumps(csp_scan))

sleep(2)
print(csp_subarray_leaf_node_dp.cspSubarrayObsState)

{'interface': 'https://schema.skao.int/ska-sdp-scan/0.4', 'scan_id': 1}
sdpSubarrayObsState.SCANNING
{'interface': 'https://schema.skao.int/ska-csp-scan/2.2', 'scan_id': 1}
cspSubarrayObsState.SCANNING


### END SCAN


DO NOT USE FOR NOW SINCE TMC SUBARRAY IS STUCK IN CONFIGURING

In [ ]:
# print("Running the End Scan command: subarray obsstate should go to Ready state")

# tmc_subarray.EndScan()

# sleep(2)
# print(f"\nTMC Subarray Observation State: {tmc_subarray.obsState}")

TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING

In [14]:
print("Running the End Scan command: subarray obsstate should go to Ready state")

sdp_subarray_leaf_node_dp.EndScan()
sleep(2)
csp_subarray_leaf_node_dp.EndScan()
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

Running the End Scan command: subarray obsstate should go to Ready state
SDP Subarray Observation State: 4
CSP Subarray Observation State: 5


### END

DO NOT USE (TMC STUCK IN CONFIGURING)

In [ ]:
# print("Running the End command: subarray obsstate should go to Idle state")

# tmc_subarray.End()

# sleep(2)
# print(f"\TMC Subarray Observation State: {cbf_subarray.obsState}")

TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING

In [15]:
print("Running the End command: subarray obsstate should go to Idle state")

sdp_subarray_leaf_node_dp.End()
sleep(2)
csp_subarray_leaf_node_dp.End()

sleep(2)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

Running the End command: subarray obsstate should go to Idle state
SDP Subarray Observation State: 2
CSP Subarray Observation State: 9


### Release Resources

DO NOT USE (TMC STUCK IN CONFIGURING)

In [ ]:
# print(
#     "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
# )

# tmc_subarray.ReleaseAllResources()

# sleep(2)
# print(f"TMC Subarray Observation State: {tmc_subarray.obsState}")

TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING

In [ ]:
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
)

sdp_subarray_leaf_node_dp.ReleaseAllResources()
sleep(2)
csp_subarray_leaf_node_dp.ReleaseAllResources()

sleep(2)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

### Telescope Off

In [ ]:
print("Running the TelescopeOff command")

tmc_central_node.TelescopeOff()

sleep(5)
print(f"TMC Central Node State: {tmc_central_node.State()}")
print(f"CSP Control State: {csp_control.State()}")
print(f"CBF Controller State: {cbf_controller.State()}")